<a href="https://colab.research.google.com/github/frank-morales2020/Cloud_curious/blob/master/MOVIEAIAGENT_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DATA SET

In [12]:
#!rm -rf /root/.cache/kagglehub/

In [13]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rounakbanik/the-movies-dataset")

print("Path to dataset files:", path)

!cp /root/.cache/kagglehub/datasets/rounakbanik/the-movies-dataset/versions/7/movies_metadata.csv /content/

Path to dataset files: /root/.cache/kagglehub/datasets/rounakbanik/the-movies-dataset/versions/7


In [14]:
import pandas as pd

# Read the CSV file into a DataFrame
movies_df = pd.read_csv("movies_metadata.csv", low_memory=False)
movies_df

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


## TRAINING

In [4]:
!pip install scikit-learn -q
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
#from sklearn.svm import SVC  # Remove the import of SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm import tqdm
from sklearn.multiclass import OneVsRestClassifier # Import OneVsRestClassifier
from sklearn.svm import SVC # Import SVC for SVM
from sklearn.impute import SimpleImputer
from sklearn.naive_bayes import MultinomialNB # Import MultinomialNB


# Used to securely store your API key
from google.colab import userdata

import google.generativeai as genai

# Set your API key
GOOGLE_API_KEY = userdata.get("GEMINI")
genai.configure(api_key=GOOGLE_API_KEY)

# Specify the model name for Gemini 2.0 Flash
model_name = "models/gemini-2.0-flash"
gemini_model = genai.GenerativeModel(model_name)

# Load the MovieLens dataset (replace with actual path)
movies_metadata = pd.read_csv("movies_metadata.csv", low_memory=False)

# Extract features and labels
features = movies_metadata[["budget", "popularity", "vote_average"]]

# Refine genre classification using MultiLabelBinarizer
mlb = MultiLabelBinarizer()
movies_metadata["genre_list"] = movies_metadata["genres"].apply(lambda x: [genre["name"] for genre in eval(x)])
labels = mlb.fit_transform(movies_metadata["genre_list"])

# Convert budget and popularity to numeric, using.loc
features.loc[:, "budget"] = pd.to_numeric(features["budget"], errors="coerce")
features.loc[:, "popularity"] = pd.to_numeric(features["popularity"], errors="coerce")

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=42
)

# Impute missing values using SimpleImputer before training
imputer = SimpleImputer(strategy='mean') # Replace NaNs with the mean of the column
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Train and evaluate the classifiers
classifiers_max = {
    "Random Forest": RandomForestClassifier(),
    "SVM": OneVsRestClassifier(SVC()), # Wrap SVC with OneVsRestClassifier
    "KNN": KNeighborsClassifier(),
    "Naive Bayes": OneVsRestClassifier(MultinomialNB()), # Use MultinomialNB for multi-label and wrap with OneVsRestClassifier
    # Wrap LogisticRegression with OneVsRestClassifier for multi-label classification
    "Logistic Regression": OneVsRestClassifier(LogisticRegression()),
    "Decision Tree": DecisionTreeClassifier(),
    "Gradient Boosting": GradientBoostingClassifier()
}

# Train and evaluate the classifiers
classifiers = {
    "Random Forest": RandomForestClassifier(),
    "SVM": OneVsRestClassifier(SVC()), # Wrap SVC with OneVsRestClassifier
    "KNN": KNeighborsClassifier(),
}

results = {}
for name, clf in tqdm(classifiers.items(), desc="Training and evaluating classifiers"):
    # Train and evaluate the classifier
    print('\n')
    #print(f"Training and evaluating {name}...")
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = accuracy  # Normalize accuracy to be between 0 and 1
    print('\n')
    print(f"{name} Accuracy: {accuracy}")




# Select the best model based on accuracy
best_model_name = max(results, key=results.get)  # Get the name of the best model
best_classifier_model = classifiers[best_model_name]  # Get the classifier object
print('\n\n')
# Use the name of the best model to access accuracy in results
print(f"Best model: {best_model_name} with accuracy {results[best_model_name]}")


Training and evaluating classifiers:   0%|          | 0/3 [00:00<?, ?it/s]

Training and evaluating classifiers:  33%|███▎      | 1/3 [00:29<00:58, 29.42s/it]



Random Forest Accuracy: 0.057180558610072575




Training and evaluating classifiers:  67%|██████▋   | 2/3 [03:17<01:50, 110.84s/it]



SVM Accuracy: 0.058060259511766




Training and evaluating classifiers: 100%|██████████| 3/3 [03:25<00:00, 68.56s/it]



KNN Accuracy: 0.0876402023312074



Best model: KNN with accuracy 0.0876402023312074


In [5]:
print("Classifier Results:")
for name, accuracy in results.items():
    print(f"{name}: Accuracy = {accuracy}")

print('\n')
# Select the best model based on accuracy
best_model = max(results, key=results.get)
print(f"Best model: {best_model} with accuracy {results[best_model]}")

Classifier Results:
Random Forest: Accuracy = 0.057180558610072575
SVM: Accuracy = 0.058060259511766
KNN: Accuracy = 0.0876402023312074


Best model: KNN with accuracy 0.0876402023312074


In [6]:
# Select the best model based on accuracy
best_model_name = max(results, key=results.get)  # Get the name of the best model
best_classifier_model = classifiers[best_model_name]  # Get the classifier object

# Use the name of the best model to access accuracy in results
print(f"Best model: {best_model_name} with accuracy {results[best_model_name]}")


Best model: KNN with accuracy 0.0876402023312074


## AGENT

In [4]:
def generate_text_with_gemini(prompt):
    """
    Generates text using the Gemini API.

    Args:
        prompt: The input prompt for text generation.

    Returns:
        The generated text from Gemini 2.0.
    """

    # Use the GenerativeModel instance to generate text
    # Pass the prompt as a positional argument instead of a keyword argument
    response = model.generate_content(prompt)
    return response.text  # Access the generated text for text models like gemini-2.0-flash



In [7]:
!pip install google-generativeai -q
!pip install backoff -q

model_name = "gemini-2.0-flash"
gemini_model = genai.GenerativeModel(model_name)

import time
import backoff  # Import backoff for retrying

In [22]:
!pip install google-generativeai -q
!pip install backoff -q
import google.generativeai as genai
import time
import backoff
import pandas as pd
import warnings

# Assuming you have set your API key using `genai.configure(api_key=YOUR_API_KEY)`

model_name = "gemini-2.0-flash"
gemini_model = genai.GenerativeModel(model_name)

class MovieGenreAgent:
    def __init__(self, model, classifier_model, gemini_model):
        self.model = model
        self.classifier_model = classifier_model
        self.gemini_model = gemini_model

    def observe(self, movie_data):
        #self.features = self.extract_features(movie_data)

        self.features = self.extract_features(movie_data)
        self.movie_title = movie_data.get("title", "Unknown")  # Extract movie title if available


    #@backoff.on_exception(backoff.expo, (ConnectionError, genai.errors.GenerativeAIError), max_tries=5, jitter=backoff.full_jitter)  # Retry on ConnectionError and GenerativeAIError
    def orient(self):
        if self.features["budget"] > 10000000:
            self.budget_level = "high"
        else:
            self.budget_level = "low"

        if self.features["popularity"] > 10:
            self.popularity_level = "high"
        else:
            self.popularity_level = "low"

        keywords = self.features['keywords']
        if isinstance(keywords, str):
            keywords = keywords.replace('[', '').replace(']', '').replace("'", '').split(', ')
        else:
            keywords = []
        keywords_string = ', '.join(keywords)

        time.sleep(2)  # Increased sleep time
        genre_analysis = self.gemini_model.generate_content(
            f"Analyze the following keywords for genre classification: {keywords_string}"
        ).text
        self.dominant_genre = genre_analysis.split()

    def decide(self):
        # Predict the genre using the selected model
        # Select only the features used during training
        prediction_features = self.features.copy()  # Create a copy to avoid modifying the original features

        # Ensure all feature values are hashable (e.g., convert lists to tuples)
        for key, value in prediction_features.items():
            if isinstance(value, list):
                prediction_features[key] = tuple(value)
            elif isinstance(value, str) and (value.startswith('[') and value.endswith(']')):
                try:
                    prediction_features[key] = tuple(eval(value))
                except (SyntaxError, NameError):
                    prediction_features[key] = float('nan')  # Or a suitable default
            elif isinstance(value, str):
                try:
                    prediction_features[key] = float(value)
                except ValueError:
                    pass  # If it's not a float, leave it as is

        # Create a new dictionary with only the desired features and hashable values
        # This avoids potential issues when filtering with a list of keys
        filtered_features = {
            key: value
            for key, value in prediction_features.items()
            if key in ['budget', 'popularity', 'vote_average'] and isinstance(value, (int, float))
        }

        #self.predicted_genre = self.classifier_model.predict(pd.DataFrame([filtered_features]))

        # Suppress the UserWarning:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=UserWarning)
            self.predicted_genre = self.classifier_model.predict(pd.DataFrame([filtered_features]))


        predicted_labels = mlb.inverse_transform(self.predicted_genre)
        #print(f"Predicted genre labels: {predicted_labels}")  # Print the labels instead of the array
        self.predicted_labels = predicted_labels  # Store predicted_labels as an attribute

    def act(self):
        # Communicate the predicted genre, potentially using Gemini 2.0
        #self.communicate_prediction()

        # Communicate the predicted genre
        #print(f"Predicted genre labels: {self.predicted_labels}")  # Print predicted labels

        print(f"Movie: {self.movie_title}")  # Print movie title

        if not self.predicted_labels[0]:
            print("Predicted genre labels: [('Other',)]")
            self.predicted_genre = [('Other',)]  # For consistent description generation
        else:
            print(f"Predicted genre labels: {self.predicted_labels}")
            self.predicted_genre = self.predicted_labels  # Store the multi-labels

        self.communicate_prediction()  # Call to generate description
        print("-" * 20)

    def extract_features(self, movie_data):
        """
        Extract features from movie data.

        Args:
            movie_data: A dictionary containing movie data.

        Returns:
            A dictionary of extracted features.
        """

        # Extract features from movie data
        # Example: Extract budget, popularity, and vote_average
        budget = movie_data["budget"]
        popularity = movie_data["popularity"]
        vote_average = movie_data["vote_average"]

        # Example: Use Gemini 2.0 to analyze the overview and extract keywords
        overview = movie_data["overview"]
        keywords = self.generate_text_with_gemini(
            f"Extract keywords from the following movie overview: {overview}"
        )
        # Process the response to get keywords
        # Example: Split the keywords by comma and strip whitespace
        keywords = [keyword.strip() for keyword in keywords.split(",")]

        # Example: Use Gemini 2.0 to analyze the overview
        sentiment = self.generate_text_with_gemini(
            f"Analyze the sentiment of the following movie overview: {overview}"
        )
        humor = self.generate_text_with_gemini(
            f"Detect humor in the following movie overview: {overview}"
        )

        features = {
            "budget": budget,
            "popularity": popularity,
            "vote_average": vote_average,
            "sentiment": sentiment,
            "humor": humor,
            # Convert keywords list to a string representation
            "keywords": ', '.join(keywords),
            #... other features...
        }
        return features

    @backoff.on_exception(
        backoff.expo, (ConnectionError, Exception), max_tries=5, jitter=backoff.full_jitter
    )

    def generate_text_with_gemini(self, prompt):
        """
        Generates text using the Gemini API with error handling and retries.

        Args:
            prompt: The input prompt for text generation.

        Returns:
            The generated text from Gemini or an error message.
        """
        try:
            response = self.gemini_model.generate_content(prompt)
            return response.text
        except Exception as e:
            print(f"Error generating text with Gemini: {e}")
            return "Error: Could not generate description."  # Or handle the error in another way


    def communicate_prediction(self):
        """
        Communicates the predicted genres and their descriptions.
        """
        genre_descriptions = []
        for genre_tuple in self.predicted_genre:
            for genre in genre_tuple:
                time.sleep(2)  # Add delay to avoid hitting rate limits
                description = ""
                chunk_prompts = [
                    f"Provide a brief overview of the movie genre: {genre}",
                    f"Describe the typical themes found in {genre} movies",
                    f"What are the common characteristics of {genre} films?",
                ]
                for chunk_prompt in chunk_prompts:
                    chunk_description = self.generate_text_with_gemini(chunk_prompt)
                    description += chunk_description + "\n"

                # Truncate description after 100 words (adjust as needed)
                description = " ".join(description.split()[:100]) + "..."

                genre_descriptions.append(f"{genre}: {description}")

        final_description = "\n".join(genre_descriptions)
        print(f"Description: {final_description}")




## TEST CASE

In [23]:
# Example movie data 1
movie_data_1 = {
    "budget": 50000000,
    "popularity": 12.5,
    "vote_average": 7.8,
    "overview": "A detective investigates a series of murders.",
    "title": "Movie Title 1"
}

# Example movie data 2
movie_data_2 = {
    "budget": 10000000,
    "popularity": 6.3,
    "vote_average": 6.5,
    "overview": "A hilarious group of friends go on a camping trip.",
    "title": "Movie Title 2"
}

# Example movie data 3
movie_data_3 = {
    "budget": 200000000,
    "popularity": 25.8,
    "vote_average": 8.5,
    "overview": "A young woman falls in love with a vampire.",
    "title": "Movie Title 3"
}

# Example movie data 4
movie_data_4 = {
    "budget": 75000000,
    "popularity": 18.2,
    "vote_average": 7.1,
    "overview": "A scientist invents a time machine.",
    "title": "Movie Title 4"
}

# Example movie data 5
movie_data_5 = {
    "budget": 15000000,
    "popularity": 9.7,
    "vote_average": 6.9,
    "overview": "A family goes on a road trip across America.",
    "title": "Movie Title 5"
}

In [24]:
# Create an instance of the agent with the best classifier model and Gemini model
agent = MovieGenreAgent(best_model, best_classifier_model, gemini_model)

# Use the agent to predict the genre for each example
for i, movie_data in enumerate([movie_data_1, movie_data_2, movie_data_3, movie_data_4, movie_data_5]):
    print(f"Processing movie data {i+1}")
    agent.observe(movie_data)
    agent.orient()
    agent.decide()
    agent.act()
    print("-" * 20)

Processing movie data 1
Movie: Movie Title 1
Predicted genre labels: ('Comedy',)
Description: C: Okay, the movie genre "C" is a bit of a tricky one because it's not a formally recognized or universally accepted genre. However, based on common usage and association, here's what "C" typically refers to in the context of movies: **"C" typically signifies a low-budget, often exploitative, and generally poorly made film.** Here's a breakdown of what that usually entails: * **Low Budget:** "C" movies almost always have extremely limited funding. This is evident in the quality of the production, special effects, acting, and overall polish. * **Exploitation Elements:** They often lean heavily on exploitation tropes to attract viewers. This...
o: It seems like you've started to type a movie genre, but haven't finished. To give you a proper overview, I need you to complete the word. However, I can offer some possibilities and overviews based on common genres starting with "O": **Possible Genres 

KeyboardInterrupt: 